# Food Hunt in downtown Toronto (Part 2)

# Load Packages

In [1]:
!pip install folium

     |████████████████████████████████| 102kB 11.2MB/s ta 0:00:01


In [2]:
!pip install pgeocode

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim

In [4]:
def locator(df, postal_code, country_code):
    import pgeocode
    import pandas

    nomi = pgeocode.Nominatim(country_code)
    df['lat'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).latitude)
    df['lng'] = df[postal_code].apply(lambda x:nomi.query_postal_code(x).longitude)

    return df

In [5]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')
wikitable = soup.find('table')
columns = [i.text.strip() for i in wikitable.find_all('th')]
data = [[j.text.strip() for j in i.find_all('td')] for i in wikitable.find_all("tr")[1:]]
trtdata = pd.DataFrame(data, columns = columns)
trtdata['Neighborhood'] = trtdata.Neighborhood.apply(lambda x: x.replace(' /',','))
trtdata.columns = ['PostalCode','Borough', 'Neighborhood']

In [6]:
trtdata1 = trtdata.loc[trtdata.Borough != 'Not assigned'].reset_index(drop=True)
trtdata1.loc[trtdata1.Neighborhood == 'Not assigned', 'Neighborhood'] = trtdata['Borough']
trtdata2 = trtdata1.groupby(['PostalCode', 'Borough']).agg(', '.join).reset_index()
trtdata3 = locator(trtdata2, 'PostalCode', 'ca')
trtdata3 = trtdata3.dropna()

In [7]:
trtdata3.head()

,PostalCode,Borough,Neighborhood,lat,lng
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
radius=1000
LIMIT=10
VERSION = '20200605'
lat = trtdata3.lat[0]
lng = trtdata3.lng[0]

In [53]:
trtdata3.head()

,PostalCode,Borough,Neighborhood,lat,lng
0,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.1930
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866
3,M1G,Scarborough,Woburn,43.7712,-79.2144
4,M1H,Scarborough,Cedarbrae,43.7686,-79.2389


In [24]:
for row in range(trtdata3.shape[0]):
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        trtdata3.lat[row], 
        trtdata3.lng[row], 
        radius, 
        LIMIT)

    # make the GET request
    result = requests.get(url).json()
    
    results.append(result)

In [25]:
clean_data = results["response"]['groups'][0]['items']

In [38]:
clean_data[0]['venue']['name']

'Canadiana exhibit'

In [39]:
venues_list=[]

In [48]:
venues_list.append([(
    v['venue']['name'], 
    v['venue']['location']['lat'], 
    v['venue']['location']['lng'],  
    v['venue']['categories'][0]['name']) for v in clean_data])

In [61]:
[item for item in venues_list[0]]

[('Canadiana exhibit', 43.81796218928876, -79.19337359666939, 'Zoo Exhibit'),
 ('Wendy’s', 43.80744841934756, -79.19905558052072, 'Fast Food Restaurant'),
 ('Upper Rouge Trail', 43.809988, -79.186147, 'Trail'),
 ('Grizzly Bear Exhibit',
  43.81703052481388,
  -79.19345807605157,
  'Zoo Exhibit'),
 ('FASTSIGNS', 43.8078818, -79.2019677, 'Business Service'),
 ('Cheetah Exhibit', 43.817588, -79.187206, 'Other Great Outdoors'),
 ('simba safari lodge', 43.81730484815479, -79.1859038113143, 'Zoo Exhibit'),
 ('Zebra Exhibit', 43.81804455844179, -79.18705579898379, 'Zoo Exhibit'),
 ('White Rhinoceros exhibit',
  43.81743351153191,
  -79.18547444686472,
  'Zoo Exhibit'),
 ('Olive Baboon Exhibit',
  43.81854541587957,
  -79.18723762523135,
  'Zoo Exhibit')]

In [59]:
venues_list

[[('Canadiana exhibit', 43.81796218928876, -79.19337359666939, 'Zoo Exhibit'),
  ('Wendy’s', 43.80744841934756, -79.19905558052072, 'Fast Food Restaurant'),
  ('Upper Rouge Trail', 43.809988, -79.186147, 'Trail'),
  ('Grizzly Bear Exhibit',
   43.81703052481388,
   -79.19345807605157,
   'Zoo Exhibit'),
  ('FASTSIGNS', 43.8078818, -79.2019677, 'Business Service'),
  ('Cheetah Exhibit', 43.817588, -79.187206, 'Other Great Outdoors'),
  ('simba safari lodge', 43.81730484815479, -79.1859038113143, 'Zoo Exhibit'),
  ('Zebra Exhibit', 43.81804455844179, -79.18705579898379, 'Zoo Exhibit'),
  ('White Rhinoceros exhibit',
   43.81743351153191,
   -79.18547444686472,
   'Zoo Exhibit'),
  ('Olive Baboon Exhibit',
   43.81854541587957,
   -79.18723762523135,
   'Zoo Exhibit')]]